In [0]:
#https://github.com/Zackory/Keras-MNIST-GAN/blob/master/mnist_dcgan.py
import os
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers.core import Reshape, Dense, Dropout, Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D, UpSampling2D
from keras.datasets import mnist
from keras.optimizers import Adam
from keras import backend as K
from keras import initializers

K.set_image_dim_ordering('th')

#random 설정
np.random.seed(1000)

# 출처에서는 100에서 성능이 더 좋았다고합니다.
# GAN과 동일한 조건을 주기 위해서 1000으로 두고 코드를 작성했다고 합니다.
randomDim = 100

#Mnist 정보를 가져오기.

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = (x_train.astype(np.float32)-127.5)/127.5
x_train = x_train[:, np.newaxis, :, :]

Using TensorFlow backend.


In [0]:
#set optimizer 
adam = Adam(lr=0.0002, beta_1=0.5)

#set the Generator layer

G= Sequential()
G.add(Dense(128*7*7, input_dim=randomDim, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
G.add(LeakyReLU(0.2))
G.add(Reshape((128, 7, 7)))
G.add(UpSampling2D(size=(2, 2)))
G.add(Conv2D(64, kernel_size=(5, 5), padding='same'))
G.add(LeakyReLU(0.2))
G.add(UpSampling2D(size=(2, 2)))
G.add(Conv2D(1, kernel_size=(5, 5), padding='same', activation='tanh'))
G.compile(loss='binary_crossentropy', optimizer=adam)


# Discriminator
D = Sequential()
D.add(Conv2D(64, kernel_size=(5, 5), strides=(2, 2), padding='same', input_shape=(1, 28, 28), kernel_initializer=initializers.RandomNormal(stddev=0.02)))
D.add(LeakyReLU(0.2))
D.add(Dropout(0.3))
D.add(Conv2D(128, kernel_size=(5, 5), strides=(2, 2), padding='same'))
D.add(LeakyReLU(0.2))
D.add(Dropout(0.3))
D.add(Flatten())
D.add(Dense(1, activation='sigmoid'))
D.compile(loss='binary_crossentropy', optimizer=adam)


# GAN = Generator + Discriminator
D.trainable = False
ganInput = Input(shape=(randomDim,))
x = G(ganInput)
ganOutput = D(x)
gan = Model(inputs=ganInput, outputs=ganOutput)
gan.compile(loss='binary_crossentropy', optimizer=adam)

dLosses = []
gLosses = []

# 하위 항목은 GAN 과 유사함
# Plot the loss from each batch
def plotLoss(epoch):
    plt.figure(figsize=(10, 8))
    plt.plot(dLosses, label='Discriminitive loss')
    plt.plot(gLosses, label='Generative loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('images/dcgan_loss_epoch_%d.png' % epoch)

# Create a wall of generated MNIST images
def plotGeneratedImages(epoch, examples=100, dim=(10, 10), figsize=(10, 10)):
    noise = np.random.normal(0, 1, size=[examples, randomDim])
    generatedImages = G.predict(noise)

    plt.figure(figsize=figsize)
    for i in range(generatedImages.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generatedImages[i, 0], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('images/dcgan_generated_image_epoch_%d.png' % epoch)

# Save the generator and discriminator networks (and weights) for later use
def saveModels(epoch):
    G.save('models/dcgan_generator_epoch_%d.h5' % epoch)
    D.save('models/dcgan_discriminator_epoch_%d.h5' % epoch)

def train(epochs=1, batchSize=128):
    batchCount = (int)(x_train.shape[0] / batchSize)
    print ('Epochs:', epochs)
    print ('Batch size:', batchSize)
    print ('Batches per epoch:', batchCount)

    for e in range(1, epochs+1):
        print ('-'*15, 'Epoch %d' % e, '-'*15)
        for _ in tqdm(range(batchCount)):
            # Get a random set of input noise and images
            noise = np.random.normal(0, 1, size=[batchSize, randomDim])
            imageBatch = x_train[np.random.randint(0, x_train.shape[0], size=batchSize)]

            # Generate fake MNIST images
            generatedImages = G.predict(noise)
            X = np.concatenate([imageBatch, generatedImages])

            # Labels for generated and real data
            yDis = np.zeros(2*batchSize)
            # One-sided label smoothing
            yDis[:batchSize] = 0.9

            # Train discriminator
            D.trainable = True
            dloss = D.train_on_batch(X, yDis)

            # Train generator
            noise = np.random.normal(0, 1, size=[batchSize, randomDim])
            yGen = np.ones(batchSize)
            D.trainable = False
            gloss = gan.train_on_batch(noise, yGen)

        # Store loss of most recent batch from this epoch
        dLosses.append(dloss)
        gLosses.append(gloss)

        if (e>50 and e == 1 or e % 5 == 0) or e%20==1:
            plotGeneratedImages(e)
            saveModels(e)

    # Plot losses from every epoch
    plotLoss(e)

if __name__ == '__main__':
    train(200)

    
    